# 8. Fixed transforms

:::{note}
You are viewing a working draft with expected completion in early 2025.
:::

The current section until this point has been focused on mapping collections of images and meta-data in whatever format is needed and wherever they may be. However, we have always assumed until now that the images returned from the dataset are in "perfect condition" and harmonized within each other. 

Unfortunately, that is rarely the case. We often work with images that have a variety of imperfections which we must correct after reading them in from their raw form. This can include images having different shapes or orientations from one other, having unnormalized intensity values with outliers, or label images not being in the one-hot format needed for our AI models.

You can call these types of preprocessing steps as "fixed transforms". They are fixed because they are deterministic and will give the same result regardless of how many times they're applied to the image. As you'll learn here, these fixed transforms should be applied when mapping datasets.

## Prerequisites

In this chapter, we will specifically use the transforms modules from nitrain along with ants and numpy to create images. We will only work with images already loaded in memory in this chapter, but keep in mind that there would be no difference if you use other types of readers.

In [2]:
import nitrain as nt
from nitrain import transforms as tx
import ants
import numpy as np

## Individual transforms

We will start by learning how to apply transforms to images individually. Let's start with a basic classification example where all of the images in our dataset are of different shape. 

In [5]:
# create data in memory
images = [ants.from_numpy(np.ones((np.random.randint(40,50),np.random.randint(40,50)))) * i for i in range(10)]
labels = [i for i in range(10)]

print('Image shapes:')
print([image.shape for image in images])

Image shapes:
[(40, 44), (44, 44), (49, 49), (45, 48), (49, 47), (48, 44), (43, 44), (42, 41), (41, 41), (49, 46)]


In that case, we may want to resample them to a common shape in order to eventually generate batches with them. There is a transform called `Resample` that takes care of this perfectly. Here is how you would apply it in the context of a dataset.

In [9]:
dataset = nt.Dataset(inputs=images, 
                     outputs=labels,
                     transforms={
                         'inputs': tx.Resample((40,40), use_voxels=True)
                     })

print([x.shape for x, y in dataset])

[(40, 40), (40, 40), (40, 40), (40, 40), (40, 40), (40, 40), (40, 40), (40, 40), (40, 40), (40, 40)]


As we see, the `transforms` argument is a dictionary where the key is a label telling the dataset which value to apply the transform to, and the value is an initialized transform. 

### Multiple transforms

Because the `transforms` argument is a dictionary, we can apply transforms to both the inputs and the outputs. A clear example of this would be when both your inputs and outputs are images.

In [15]:
dataset = nt.Dataset(inputs=images, 
                     outputs=images,
                     transforms={
                         'inputs': tx.Resample((40,40), use_voxels=True),
                         'outputs': tx.Resample((40,40), use_voxels=True)
                     })

x, y = dataset[0]
print(x.shape, '|', y.shape)

(40, 40) | (40, 40)


### Composing transforms

Rarely will we apply only one transform to an image. Thankfully, composing transforms in nitrain is as simple as combining them in lists. Here is an example where we resample the image and then take the square root of the image.

In [14]:
dataset = nt.Dataset(inputs=images, 
                     outputs=labels,
                     transforms={
                         'inputs': [tx.Resample((40,40), use_voxels=True),
                                    tx.Sqrt()]
                     })

x, y = dataset[4]
print('Shape:', x.shape, ' | Mean: ', x.mean())

Shape: (40, 40)  | Mean:  2.0


When transforms are composed in lists, they will be applied in the order given. This is useful for when certain transforms require some other preprocessing to occur before working correctly.

## Multi-image transforms

Until now, you've seen how to apply transforms to individual images. But there are certain transforms that require multiple images or simply work more efficiently. You can apply transforms to multiple images together by having the transforms dictionary key be a tuple of the value labels. 

The example with resampling the images as inputs and outputs would therefore more appropriately look like this.

In [20]:
dataset = nt.Dataset(inputs=images, 
                     outputs=images,
                     transforms={
                         ('inputs', 'outputs'): tx.Resample((40,40), use_voxels=True),
                     })

x, y = dataset[0]
print(x.shape, '|', y.shape)

(40, 40) | (40, 40)


This concept will be particularly important later on in the book when learning about random, augmenting transforms. Such random transforms have to be applied together in many cases to ensure the alignment between, say, an image and its segmentation is maintained.

## Adding labels

We've only used the keys `inputs` and `outputs` in our transforms dictionary. But in the case of multiple readers as inputs or outputs, you may want to apply a transform to only one of the readers. This is possible by using (or at least understanding) the way readers are labelled. 

For example, if you have two inputs then there are three available labels: "inputs", "inputs-0", and "inputs-1". Supplying a transform with "inputs" as key will apply the transform to both inputs, while "inputs-0" or "inputs-1" as key will apply the transform to only the first or second input, respectively. Here is an example:

In [34]:
# resample each input differently
dataset = nt.Dataset(inputs=[images, images], 
                     outputs=images,
                     transforms={
                         'inputs-0': tx.Resample((40,40), use_voxels=True),
                         'inputs-1': tx.Resample((50,50), use_voxels=True),
                     })

x, y = dataset[0]

print('Input 1 shape: ', x[0].shape, ' | Input 2 shape:', x[1].shape)

Input 1 shape:  (40, 40)  | Input 2 shape: (50, 50)


However, it is possible to use custom labels by passing in your inputs as a dictionary instead of a list, where the labels of each input are the key.

In [36]:
# resample each input differently
dataset = nt.Dataset(inputs={'first-image': images, 
                             'second-image': images}, 
                     outputs=images,
                     transforms={
                         'first-image': tx.Resample((40,40), use_voxels=True),
                         'second-image': tx.Resample((50,50), use_voxels=True),
                     })

x, y = dataset[0]

print('first-image: ', x[0].shape, ' | second-image:', x[1].shape)

first-image:  (40, 40)  | second-image: (50, 50)


This allows you to supply custom labels to your inputs and outputs that better represent what the values actually are. Nonetheless, the authomatic labels are always available.

## Custom functions

The nitrain library has a large collection of transforms and we are happy to consider adding more. But if you don't see the transform that you need, or if you want to do something unique to your images, you can use the `tx.CustomFunction` transform to pass in a user-defined function to be applied as a transform.

In [41]:
def make_string(value):
    return f'class-{value}'

# resample each input differently
dataset = nt.Dataset(inputs=images,
                     outputs=labels,
                     transforms={
                         'outputs': tx.CustomFunction(make_string)
                     })

x, y = dataset[0]
print(y)

class-0


## Summary

In this chapter, you learned about applying fixed transforms to your datasets. With nitrain transforms, it's possible to apply transforms in arbitrary order and to multiple images at once. It's important to identify deterministic, preprocessing transforms and distinguish them from random, augmentation transforms (which you'll learn about later) because we can cache the preprocessed images to speed things up. 

This chapter focused on how to apply transforms and did not give a full rundown of all the available transforms. To understand which transforms are available in nitrain, you should check out the [transforms](www.github.com/nitrain/nitrain/transforms) folder on the GitHub page.